In [94]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [95]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [96]:
root = "/data/gpfs/home/tfaske/d/rabbit/full"

In [97]:
snp_dir = os.path.join(root,'SNPcall')

In [98]:
bwa_dir = os.path.join(snp_dir,'bwa')

In [99]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa


In [100]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [101]:
len(bam_files), bam_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bwa/EN_AH_10_sorted.bam')

In [9]:
samtools = "samtools"
bcftools = "bcftools"
java = "java"
perl = "perl"
#perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

In [10]:
assembly = "/data/gpfs/assoc/denovo/tfaske/rabbit/full/assembly/reference.fasta"

#indexed done by dDocent 

In [11]:
!$samtools faidx {assembly}

# Calculates coverage from bam_files
also outputs a file called "bam_coverage.csv" in the bam_coverage folder  

this will make a file for each sorted.bam and then consolidate in order to run efficiently

first run to get size and remove empties: ls --sort=size -lh *sorted_bam 

In [27]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall


In [28]:
!mkdir bam_coverage

mkdir: cannot create directory ‘bam_coverage’: File exists


In [29]:
!mkdir bam_coverage/shdir

In [15]:
### select options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'bam_cov'
time = '1-00:00:00' #time limit 1 days
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [32]:
def write_bamcov_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_files):
    for i in range(0,len(bam_files)):
        b = bam_files[i].split('/')[10].split('_sorted.bam')[0] #set this 
        with open("bam_coverage/shdir/run_bamcov_%s.sh" % (b), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bamcov
#SBATCH --output bam_coverage/%s_bamcov.txt
#SBATCH --mail-user=%s \n\n
    
samtools depth -a %s""" % (account,partition,time,int(cpus),int(mem_cpu),b,b,email,bam_files[i]))

In [33]:
write_bamcov_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_files)

#### finds all bamcov slurm scripts and writes bash script to sbatch them

In [34]:
bamcov_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/bam_coverage/shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        bamcov_files.append(x)
bamcov_files = sorted(bamcov_files)

In [35]:
len(bamcov_files),bamcov_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bam_coverage/shdir/run_bamcov_EN_AH_1.sh')

In [36]:
def write_bash_bamcov_sh(sh_files):
    with open("SNPcall/run_bash_bamcov.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [37]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


In [38]:
write_bash_bamcov_sh(bamcov_files)

### run_bash_bamcov.sh
    cd /data/gpfs/home/tfaske/d/rabbit/full/SNPcall 
    source activate py36
    sbatch run_bash_bamcov.sh
    
### Read in all bamcov files and summarize 

In [39]:
cov_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/bam_coverage'))
files = !find . -name '*cov.txt'
files = [os.path.abspath(x) for x in files]
for x in files:
        cov_files.append(x)
cov_files = sorted(cov_files)

In [40]:
len(cov_files),cov_files[0]

(608,
 '/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/bam_coverage/EN_AH_10_bamcov.txt')

In [77]:
bam_names = []
cov_list = []
for i in range(0,len(cov_files)):
    cov_file = cov_files[i]
    c = cov_file.split('/')[10].split('_bamcov.txt')[0] #set this
    b = c + '_sorted.bam'
    coverage = !awk '{{ total += $$3 }} END {{ print total/NR }}' "$cov_file"
    bam_names.append(b)
    cov_list.append(float(coverage[0]))
    #print(float(coverage[0]),b)
cov_df = pd.DataFrame(bam=bam_names,coverage=cov_list)
cov_df.head()

TypeError: __init__() got an unexpected keyword argument 'bam'

In [78]:
cov_dict = {"bam":bam_names,'coverage':cov_list}
cov_df = pd.DataFrame(cov_dict)
cov_df.head()

,bam,coverage
0,EN_AH_10_sorted.bam,32.7632
1,EN_AH_11_sorted.bam,30.5986
2,EN_AH_12_sorted.bam,35.1714
3,EN_AH_13_sorted.bam,33.0546
4,EN_AH_14_sorted.bam,22.6255


In [79]:
cov_out = os.path.join(snp_dir,'bam_coverage/bam_coverage.csv')
cov_df.to_csv(path_or_buf=cov_out)

In [80]:
cov_df.coverage.describe()

count    608.000000
mean      29.271217
std        6.462445
min        2.030350
25%       26.070175
50%       30.365850
75%       33.688850
max       46.151000
Name: coverage, dtype: float64

### Select good coverage bam files and move them to new dir

In [84]:
len(cov_df[cov_df.coverage < 10]),len(cov_df[cov_df.coverage < 15]),len(cov_df[cov_df.coverage < 11])

(9, 20, 10)

In [85]:
good_bam = cov_df.bam[cov_df.coverage >= 10]
good_bam = good_bam.tolist()
len(good_bam),good_bam[0]

(599, 'EN_AH_10_sorted.bam')

In [86]:
good_bam_files = []
for i in range(0,len(good_bam)):
    bam = bwa_dir + '/' + good_bam[i]
    good_bam_files.append(bam)
len(good_bam_files), good_bam_files[1]

(599, '/data/gpfs/home/tfaske/d/rabbit/full/SNPcall/bwa/EN_AH_11_sorted.bam')

In [87]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall


In [88]:
!mkdir 'good_bams'

In [89]:
good_bam_dir = os.path.join(snp_dir,'good_bams')
assert good_bam_dir

In [90]:
for i in range(0,len(good_bam_files)):
    good_bam = str(good_bam_files[i])
    !cp $good_bam $good_bam_dir

In [91]:
cd $good_bam_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall/good_bams


In [92]:
good_bam_cp_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    good_bam_cp_files.append(x)
good_bam_cp_files = sorted(good_bam_files)

In [93]:
len(good_bam_cp_files),good_bam_cp_files[0]

(599, '/data/gpfs/home/tfaske/d/rabbit/full/SNPcall/bwa/EN_AH_10_sorted.bam')

# SNP CALL

In [102]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall


In [103]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbit/full/SNPcall'

In [104]:
### select options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'samtools_ERNA'
time = '4-00:00:00' #time limit 3 day2
cpus = 64
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [105]:
def write_callsnps_sh(account,partition,jobname,time,cpus,mem_cpu,email,assembly,good_bam_dir,snp_dir):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    with open("./run_callsnps.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_callsnps
#SBATCH --output output_callsnps.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),jobname,email))

        o.write("samtools mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR -Iugf %s %s/*sorted.bam | bcftools call -f GP,GQ -vmO z -o %s/samtools1-9.%s.vcf.gz" %
            (assembly,good_bam_dir,snp_dir,jobname))

In [106]:
write_callsnps_sh(account,partition,jobname,time,cpus,mem_cpu,email,assembly,good_bam_dir,snp_dir)

# run_call_snps.sh 
    cd /data/gpfs/home/tfaske/d/rabbit/full/SNPcall
    source activate py36
    sbatch run_callsnps.sh